In [7]:
# install pycaret
!pip install git+https://github.com/pycaret/pycaret.git@master --upgrade --quiet
!pip install mlflow

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


In [28]:
#### Import Libraries ####
import pandas as pd
import numpy as np
from pycaret.regression import *
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# loading sample dataset
from pycaret.datasets import get_data
data = get_data('traffic')



,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,Rush Hour,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,1,5545
1,None,289.36,0.0,0.0,75,Clouds,0,4516
2,None,289.58,0.0,0.0,90,Clouds,0,4767
3,None,290.13,0.0,0.0,90,Clouds,0,5026
4,None,291.14,0.0,0.0,75,Clouds,0,4918


In [20]:
# Define target variable
target = 'traffic_volume'

# define X and y
X = data.drop(columns=target)
y = data[target]

# Split the data into training and testing sets with a 70-30 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

df_train = X_train.join(y_train)
df_test = X_test.join(y_test)

numeric_features = [col for col in data.columns]

Now, we’ll use the setup() function from PyCaret to prepare the dataset for further modeling.

In [9]:
pycaret_setup = setup(data=data,
                      target='traffic_volume',
                      session_id = 438
                      )

,Description,Value
0,Session id,438
1,Target,traffic_volume
2,Target type,Regression
3,Original data shape,"(48204, 8)"
4,Transformed data shape,"(48204, 29)"
5,Transformed train set shape,"(33742, 29)"
6,Transformed test set shape,"(14462, 29)"
7,Numeric features,5
8,Categorical features,2
9,Preprocess,True


PyCaret determines whether you’re performing a classification or regression analysis based on the target variable you provided during the setup process.

We know this by running compare_models() function in the next code block, which is used to display the list of available pre-trained models that can be used for the machine learning tasks suitable for the data provided.

In [10]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,1485.1216,3064337.3585,1750.4342,0.2221,0.9236,2.3875,0.2870
lightgbm,Light Gradient Boosting Machine,1502.4277,3077063.5115,1754.0547,0.2189,0.9297,2.4635,0.5040
gbr,Gradient Boosting Regressor,1530.7769,3148984.7500,1774.4289,0.2006,0.9421,2.5603,1.9090
ada,AdaBoost Regressor,1577.2863,3282090.0319,1811.5740,0.1669,0.9730,2.7074,0.4820
knn,K Neighbors Regressor,1573.7193,3662877.1551,1913.7609,0.0702,0.9602,2.5605,0.4830
rf,Random Forest Regressor,1547.0023,3740449.9683,1933.7739,0.0504,0.9554,2.4869,6.1160
omp,Orthogonal Matching Pursuit,1712.8788,3848922.2942,1961.8288,0.0229,1.0224,2.9640,0.1030
dummy,Dummy Regressor,1743.7632,3939897.8403,1984.8734,-0.0001,1.0318,2.9022,0.0990
et,Extra Trees Regressor,1679.3206,4592693.0108,2142.8898,-0.1661,1.0486,2.6657,3.5450
dt,Decision Tree Regressor,1749.2798,5217471.6829,2283.9553,-0.3246,1.1263,2.5748,0.1700


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [23]:
# Pycaret top 3 models to analyze
best_model = compare_models(include = ['rf','xgboost','lightgbm','et', 'ada'],errors='raise', n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,1485.1216,3064337.3585,1750.4342,0.2221,0.9236,2.3875,0.2880
lightgbm,Light Gradient Boosting Machine,1502.4277,3077063.5115,1754.0547,0.2189,0.9297,2.4635,1.2100
ada,AdaBoost Regressor,1577.2863,3282090.0319,1811.5740,0.1669,0.9730,2.7074,0.6140
rf,Random Forest Regressor,1547.0023,3740449.9683,1933.7739,0.0504,0.9554,2.4869,6.5520
et,Extra Trees Regressor,1679.3206,4592693.0108,2142.8898,-0.1661,1.0486,2.6657,3.9260


Processing:   0%|          | 0/27 [00:00<?, ?it/s]

Now that we’ve taken a look how the models performed on the training dataset, it’s time to make predictions on the test set and see how the top 3 models perform. The top 3 models are Extra Trees Regressor, AdaBoost Regressor, and XGBoost Regressor.

In [24]:
best_model

[XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, random_state=438, ...),
 LGBMRegressor(n_jobs=-1, random_state=438),
 AdaBoostRegressor(random_state=438)]

In [11]:
# evaluate trained model
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [12]:
# predict on hold-out/test set
pred_holdout = predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,1481.7419,3048303.0501,1745.9390,0.2312,0.9228,2.5142


In [27]:
# Assign best_model to models for code simplicity
models = best_model.copy()

# Define dictionaries to contain results
eval_results = {}
model_dict = {}

for i in range(len(models)):

    # Print model name
    model_name = models[i].__class__.__name__
    print(model_name)

    # Get predictions on the test set for the model
    predictions = predict_model(models[i], data=df_test)

    # Actual vs predicted
    y_true = df_test[target]  # Use the 'traffic_volume' column from df_test as the actual values
    y_pred = predictions[target]  # Use the 'traffic_volume' column from predictions as the predicted values

    # Calculate MAE, MSE, RMSE, and R-Squared for model evaluation
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    # Use rmse as the key indicator for best performance
    eval_results[model_name] = rmse
    model_dict[model_name] = models[i]

# Find the model with the lowest rmse
top_model = min(eval_results, key=eval_results.get)

# Print the result
print("The top performing model on the test dataset:", top_model, ", rmse:", eval_results[top_model])



XGBRegressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,1426.4490,2838221.0392,1684.7021,0.2862,0.9006,2.1547


LGBMRegressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,1477.0268,2978356.7031,1725.7916,0.2510,0.9195,2.2589


AdaBoostRegressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,1577.9558,3286798.4360,1812.9530,0.1734,0.9719,2.5411


The top performing model on the test dataset: XGBRegressor , rmse: 0.0


Now the situation has changed. When making predictions using the top 3 models based on the initial experiment based on the test set, XGBoost Regressor outperformed Extra Trees Regressor by a margin when we set RMSE as the key performance indicator. As a result, XGBoost Regressor has been selected as the top model with the lowest RMSE.

# save this model

In [29]:
!mkdir models

In [30]:
save_path = '/content/models/'
date = datetime.now().strftime("%Y-%m-%d")

# define dependent variables
model_file_name = f'{save_path}{top_model}_{date}'
filename = f'{top_model}_{date}.pkl'

# save model in local path
save_model(model_dict[top_model], model_file_name)

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['temp', 'rain_1h', 'snow_1h',
                                              'clouds_all', 'Rush Hour'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['holiday', 'weather_main'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWrapper(inclu...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_le

In [34]:
saved_model = load_model('/content/models/XGBRegressor_2023-11-02')

Transformation Pipeline and Model Successfully Loaded


In [36]:
predictions = predict_model(saved_model, data=X_test)
predictions.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,Rush Hour,prediction_label
35748,None,289.489990,0.0,0.0,1,Fog,1,4886.825684
147,None,279.350006,0.0,0.0,1,Clear,0,2467.457764
24362,None,291.390015,0.0,0.0,36,Clouds,0,1926.795166
35495,None,294.019989,0.0,0.0,90,Thunderstorm,0,2772.964600
11010,None,268.690002,0.0,0.0,64,Mist,1,4304.681641
